In [1]:
import tensorflow as tf
tf.version.VERSION

'2.3.0'

In [2]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import Model
import pandas as pd

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14568666558976158284,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8386821039229121899
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6535095513
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15698207642549563591
 physical_device_desc: "device: 0, name: GeForce RTX 2060 SUPER, pci bus id: 0000:29:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5078380158067232663
 physical_device_desc: "device: XLA_GPU device"]

In [4]:
from tensorflow.keras.applications.efficientnet import EfficientNetB6, preprocess_input
img_rows = 512
img_cols = 512
base_model = EfficientNetB6(include_top=False, weights='imagenet',input_shape=(img_rows,img_cols,3))

In [5]:
for i,layer in enumerate(vgg16.layers):
    print(str(i)+" "+layer.__class__.__name__,layer.trainable)

NameError: name 'vgg16' is not defined

In [6]:
# Layers are set to trainable as True by default
for layer in vgg16.layers:
    layer.trainable = True
    
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

NameError: name 'vgg16' is not defined

In [5]:
from tensorflow.keras import layers,models

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(5,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb6 (Functional)  (None, 16, 16, 2304)      40960143  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2304)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               590080    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1285      
Total params: 41,551,508
Trainable params: 41,327,069
Non-trainable params: 224,439
_________________________________________________________________


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Cassava_Leaf_Disease_Classificaiton/data/train'
validation_data_dir = 'Cassava_Leaf_Disease_Classificaiton/data/test'

train_datagen = ImageDataGenerator(rescale=1/255,
                                  rotation_range = 20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

train_batchsize = 2
val_batchsize = 2

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(img_rows,img_cols),
                                                   batch_size=train_batchsize,
                                                   class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                             target_size=(img_rows,img_cols),
                                                             batch_size=val_batchsize,
                                                             class_mode='categorical',
                                                             shuffle=False)

Found 19237 images belonging to 5 classes.
Found 2160 images belonging to 5 classes.


In [9]:
from tensorflow.keras.optimizers import RMSprop,SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('cassavia.h5',
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=5,
                         verbose=1,
                         restore_best_weights=True)

#reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                        #factor=0.2,
                        #patience=2,
                        #min_lr=1e-6,
                        #mode='min',
                        #verbose=1)

#We put our call backs into a callback list
callbacks = [earlystop,checkpoint]

#Note we use a very small learning rate
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.0001),
             metrics=['accuracy'])

epochs = 5
batch_size = 2

history = model.fit(train_generator,
                   steps_per_epoch=19237//batch_size,
                   epochs=epochs,
                   callbacks=callbacks,
                   validation_data = validation_generator,
                   validation_steps=2160//batch_size)

Epoch 1/5
9618/9618 [==============================] - ETA: 0s - loss: 0.6700 - accuracy: 0.7669
Epoch 00001: val_loss improved from inf to 0.86827, saving model to cassavia.h5
9618/9618 [==============================] - 3473s 361ms/step - loss: 0.6700 - accuracy: 0.7669 - val_loss: 0.8683 - val_accuracy: 0.7523
Epoch 2/5
9618/9618 [==============================] - ETA: 0s - loss: 0.4890 - accuracy: 0.8346
Epoch 00002: val_loss improved from 0.86827 to 0.60984, saving model to cassavia.h5
9618/9618 [==============================] - 3472s 361ms/step - loss: 0.4890 - accuracy: 0.8346 - val_loss: 0.6098 - val_accuracy: 0.8370
Epoch 3/5
9618/9618 [==============================] - ETA: 0s - loss: 0.4297 - accuracy: 0.8542
Epoch 00003: val_loss did not improve from 0.60984
9618/9618 [==============================] - 3470s 361ms/step - loss: 0.4297 - accuracy: 0.8542 - val_loss: 0.6243 - val_accuracy: 0.8065
Epoch 4/5
9618/9618 [==============================] - ETA: 0s - loss: 0.3852 - 

In [4]:
from tensorflow import keras
loaded_model = keras.models.load_model('cassavia.h5')

In [5]:
loaded_model

In [6]:
from keras.preprocessing import image
import numpy as np
im = pd.read_csv('C:/Users/PC/Documents/Jupyter/Kaggle/Cassava_Leaf_Disease_Classificaiton/sample_submission.csv')

In [7]:
label = []
for i in im.image_id:
    Im = image.load_img('Cassava_Leaf_Disease_Classificaiton/test/'+i,target_size=(512,512))
    Im = image.img_to_array(Im)
    Im = np.expand_dims(Im,axis=0)
    Im = Im/255
    pred = np.argmax(loaded_model.predict(Im), axis=-1)
    print(pred)
    label.append(int(pred))
my_submission = pd.DataFrame({'image_id':im.image_id,'label':label})
my_submission.to_csv('submission.csv',index=False)

[4]


In [6]:
im.image_id

0    2216849948.jpg
Name: image_id, dtype: object

In [14]:
Im = image.load_img('Cassava_Leaf_Disease_Classificaiton/test/2216849948.jpg',target_size=(512,512))